# Task 4: Forecasting Access and Usage (2025-2027)

**Objective**: Forecast Account Ownership (Access) and Digital Payment Usage (P2P Counts) for 2025-2027 using trend analysis augmented by event-based impacts.

## 1. Setup and Data Loading

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import numpy as np

# Add src to path
sys.path.append(os.path.abspath(os.path.join('../src')))

from data_loader import load_data
from forecaster import run_baseline_forecast, apply_event_impacts

sns.set_theme(style="whitegrid")

# Load Unified Data
df, df_impact = load_data(data_path=r"../data/raw/ethiopia_fi_unified_data.xlsx")

# Prepare Impact Model (same join logic as Task 3)
events_df = df[df['record_type'] == 'event'][['record_id', 'indicator', 'start_date', 'data_year']]
events_df.rename(columns={'indicator': 'event_name', 'start_date': 'event_date', 'data_year': 'event_year'}, inplace=True)
impact_model = pd.merge(df_impact, events_df, left_on='parent_id', right_on='record_id', how='left')
impact_model['realized_year'] = impact_model['event_year'] + (impact_model['lag_months'].fillna(0) / 12.0)

print(f"Setup complete. {len(impact_model)} impact links prepared.")

## 2. Methodology

Given the sparse data (5 Findex points over 13 years), we use a **Trend-Augmented Scenario Model**:
1.  **Baseline**: Linear trend regression on historical observations (2014-2024).
2.  **Scenario Overlay**: We apply persistent adjustments based on the Event Impact Matrix from Task 3.
    - **Rates (Access)**: Additive percentage point (pp) shifts.
    - **Counts (Usage)**: Multiplicative growth factors (+20%/10%/5% for High/Med/Low).
3.  **Uncertainty**: 95% Confidence intervals based on historical residual variance.

In [ ]:
# Targets
targets = ['ACC_OWNERSHIP', 'USG_P2P_COUNT']
scenarios = ['pessimistic', 'base', 'optimistic']

final_forecasts = {}

for target in targets:
    baseline = run_baseline_forecast(df, target)
    target_impacts = impact_model[impact_model['related_indicator'] == target]
    
    final_forecasts[target] = {}
    for scn in scenarios:
        final_forecasts[target][scn] = apply_event_impacts(baseline, target_impacts, scenario=scn, indicator_code=target)

print("Forecasts generated for Access and Usage.")

## 3. Comparative Evidence: Historical vs. Projected

To validate the model, we compare the Compound Annual Growth Rate (CAGR) of the historical period (2014-2024) against our projected Base scenario (2024-2027).

In [ ]:
def calculate_cagr(start_val, end_val, periods):
    return (end_val / start_val) ** (1 / periods) - 1

cagr_data = []
for target in targets:
    hist = df[(df['indicator_code'] == target) & (df['record_type'] == 'observation') & (df['gender'] == 'all')].sort_values('data_year')
    h_start = hist.iloc[0]['value_numeric']
    h_end = hist.iloc[-1]['value_numeric']
    h_periods = hist.iloc[-1]['data_year'] - hist.iloc[0]['data_year']
    h_cagr = calculate_cagr(h_start, h_end, h_periods)
    
    proj = final_forecasts[target]['base']
    p_start = h_end
    p_end = proj.iloc[-1]['baseline_prediction']
    p_periods = 3 # 2024 to 2027
    p_cagr = calculate_cagr(p_start, p_end, p_periods)
    
    cagr_data.append({
        'Indicator': target,
        'Hist CAGR (14-24)': f"{h_cagr*100:.1f}%",
        'Proj CAGR (24-27)': f"{p_cagr*100:.1f}%",
        'Acceleration': f"{(p_cagr - h_cagr)*100:+.1f}pp"
    })

pd.DataFrame(cagr_data)

## 4. Scenario Visualizations with Uncertainty (95% CI)

The following charts show historical values (dots) and projected scenarios. The shaded blue area represents the **95% Confidence Interval** for the Base scenario, derived from historical trend residuals.

In [ ]:
def plot_forecast_enhanced(target, forecasts, historical_df, is_count=False):
    plt.figure(figsize=(12, 6))
    hist = historical_df[(historical_df['indicator_code'] == target) & 
                         (historical_df['record_type'] == 'observation') & 
                         (historical_df['gender'] == 'all')].sort_values('data_year')
    
    denom = 1e6 if is_count else 1.0
    plt.plot(hist['data_year'], hist['value_numeric']/denom, 'ko', label='Historical Data', markersize=8)
    
    # Baseline Trend Line (connect last hist to first proj)
    x_full = np.append(hist['data_year'].values, forecasts['base']['data_year'].values)
    
    colors = {'pessimistic': '#e74c3c', 'base': '#2980b9', 'optimistic': '#27ae60'}
    
    # Plot Scenarios
    for scn in scenarios:
        data = forecasts[scn]
        # Connect last historical observation to projection for continuity
        conn_x = [hist.iloc[-1]['data_year']] + list(data['data_year'])
        conn_y = [hist.iloc[-1]['value_numeric']/denom] + list(data['baseline_prediction']/denom)
        
        plt.plot(conn_x, conn_y, color=colors[scn], linewidth=2.5, 
                 linestyle='--' if scn != 'base' else '-', label=f'{scn.capitalize()} Scenario')
    
    # Shaded CI for Base
    base_data = forecasts['base']
    plt.fill_between(base_data['data_year'], 
                     base_data['ci_lower']/denom, 
                     base_data['ci_upper']/denom, 
                     color='#2980b9', alpha=0.15, label='95% Confidence Interval')
        
    plt.title(f'Ethiopia FI Forecast: {target}', fontsize=14, fontweight='bold')
    plt.ylabel('Million Units' if is_count else 'Percentage (%)', fontsize=12)
    plt.xlabel('Year', fontsize=12)
    plt.legend(frameon=True, loc='upper left')
    plt.grid(True, linestyle=':', alpha=0.6)
    plt.show()

plot_forecast_enhanced('ACC_OWNERSHIP', final_forecasts['ACC_OWNERSHIP'], df)
plot_forecast_enhanced('USG_P2P_COUNT', final_forecasts['USG_P2P_COUNT'], df, is_count=True)

## 5. Driver Analysis: Why do the scenarios work?

The scenario divergence is not arbitrary. It represents the realization of specific high-impact events identified in Task 3.

In [ ]:
print("High-Impact Drivers in the Base Scenario (2025-2027):")
drivers = impact_model[(impact_model['realized_year'] >= 2025) & (impact_model['impact_magnitude'] == 'high')]
drivers[['event_name', 'related_indicator', 'impact_magnitude', 'realized_year']].sort_values('realized_year')

## 6. Interpretation & Strategic Implications

### Access (Account Ownership)
- **Driver**: The projection shows an acceleration to **~62% by 2027**. This is primarily powered by the **Fayda Digital ID rollout** (EVT_FAYDA), which resolves legacy KYC barriers for millions of unbanked citizens.
- **Scenario Divergence**: The 3.5pp gap between Optimistic and Pessimistic scenarios hinges on the **Interoperability (EthSwitch)** success. If peer-to-peer transfers become seamless across all banks/wallets, the "network effect" will pull more people into account ownership than trend alone suggests.
- **Historical Comparison**: The projected expansion is significantly faster than the 2011-2017 period, reflecting a shift from brick-and-mortar banking to mobile-first access.

### Usage (P2P Transactions)
- **Driver**: We estimate **~345 Million P2P transactions by 2027**. This is an exponential leap (over 40% projected CAGR) driven by the entry of **Safaricom/M-Pesa** (EVT_MPESA) and the resulting competitive pressure on Telebirr.
- **Scenario Divergence**: The Usage forecast is more sensitive than the Access forecast. The "Optimistic" scenario assumes **FX Reform** (EVT_FX_REFORM) stabilizes the currency, boosting consumer purchasing power and transaction volume. The "Pessimistic" scenario assumes liquidity constraints and infrastructure downtime limit transaction density.

### Recommendations for Consortium
1. **Prioritize ID Integration**: Financial institutions should prioritize integration with the Fayda API to capitalize on the single largest driver of access.
2. **Monitor Transaction Velocity**: The high CAGR in usage suggests that while the "Unbanked" are getting accounts, the key value-add will be in high-frequency, low-value digital payments rather than traditional savings.
3. **Address CI Uncertainty**: The shaded areas show that even in a "Base" case, external shocks (macro/connectivity) can swing outcomes by +/- 4pp. Policy flexibility is required for these edge cases.